In [2]:
import importlib
import classification_training_utils
importlib.reload(classification_training_utils)
import utils
importlib.reload(utils)

import sys
sys.path.append('../')

import pandas as pd
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util
from sentence_transformers.datasets import SentenceLabelDataset
from sentence_transformers.readers import InputExample
from sentence_transformers.evaluation import TripletEvaluator
from sklearn.metrics.pairwise import cosine_similarity
from utils import load_model, replace_nan_with, load_big_consulting_export, callback, create_replace_no_tags_embeddings
import numpy as np
import random
from torch.utils.data import DataLoader
from datetime import datetime
import os
from collections import defaultdict
import S3BERT.src.model_freeze as freeze
import matplotlib.pyplot as plt
plt.style.use('ggplot')


from classification_training_utils import get_big_consulting_df, collect_classification_labels, get_relevant_classifications, train, filter_relevant_classifications, get_top_values, get_news_df
from transformers import EarlyStoppingCallback
import math
import torch
from tqdm import tqdm
tqdm.pandas()
import pickle

In [3]:
params = {}
params["EPOCHS"] = 4
params["USE_REPLACE_DATA"] = True
params["USE_ORIGINAL_DATA"] = False
params["UNFREEZE_LAYERS"] = 2
params["EXCLUDE_ENTITY_OTHER"] = True
params["INITIALIZED_MODEL"] = None #"brjezierski/S3BERT" # "intfloat/e5-small-v2"
params["OCCURENCE_CUTOFF"] = 2
params["CREATE_NEW_SPLIT"] = False
params["BATCH_SIZE"] = 32

dataset_dir = "../classification-training-data/" #5-9-23/"
sbert_model = load_model() if not params["INITIALIZED_MODEL"] else load_model(model=params["INITIALIZED_MODEL"])

In [4]:
prefix = "../glanos-data/embeddings/"

# with open(f'{prefix}big_consulting_2_snippet.pickle', 'rb') as f:
with open(f'{prefix}big_consulting_2_replace_no_tags.pickle', 'rb') as f:
    replace_no_tags_embeddings = pickle.load(f)

replace_data_path = '../glanos-data/datasets/big_consulting_export_replace.tsv'

if os.path.exists(replace_data_path):
    df = pd.read_csv(replace_data_path, sep='\t')
#     df['embedding'] = df['snippet'].map(replace_no_tags_embeddings)
    df = create_replace_no_tags_embeddings(df, replace_no_tags_embeddings)
    df['embedding'] = df['replace_no_tags'].map(replace_no_tags_embeddings)

df = collect_classification_labels(df, verbose=True)
df = get_top_values(df, params)
all_classifications, relevant_classifications = get_relevant_classifications(df, params)
df, classifications = filter_relevant_classifications(df, all_classifications, relevant_classifications)


['MILESTONE', '^INVEST', 'PRODUCT', '^SELL', 'CLOSE-FACILITY', 'OTHER', 'PRODUCT-STOP', 'INTERNAL', '^SUPPLY', 'RATED', 'OP_LOC', 'EMPLOY-STOP', 'INVEST', 'PROBLEM-SOLUTION', 'SUE-WIN', 'SUE-END', 'EVENT', 'TECH-PATENT', 'HIRE', 'MARKET-COVERAGE', 'COWORK', '^SERVICE', '^USE', 'SERVICE', 'MARKET', 'SUPPLY', 'AGREEMENT', 'PRODUCT-ADV', 'TECH', 'ABUSE-BR', 'PRODUCT-LAUNCH', 'PROBLEM', 'AWARD', 'GROW', 'PART_OF', 'LAUNCH', 'GROW-MARKET', 'OPERATE-LAUNCH', 'USE-PROD', 'GROW-NEUTRAL', 'PR', 'PENALTY-IP', '^EMPLOY', 'LAUNCH-FACILITY', 'DECLINE-MARKET', 'STAFFING', 'ENTITY', 'ACQUIRE-ASSET', 'ACQUIRE', 'FIGHT-ABUSE-HUM', 'CORP_REPORT', 'ABUSE-LAW', 'MARKET_LOC', 'RIGHTS', 'SELL', 'USE', 'FOUNDER', 'PARTNER', 'EMPLOY-START', 'MEDIASCORE', 'STATE', 'ACTIVITY_LOC', 'ACQUIRED', 'CRITIC', 'PARTNER_JV', 'EMPLOY', 'PENALTY', 'GROW-HIRE', 'GENERAL', 'REPORT', 'COMPETENCE', '^DEAL', '^SUE', 'AWARD-SPONS', 'CAREER', 'COMPETE', '^SELL-TO', 'STAFFING-LAYOFFS', 'LEADERSHIP', 'DECLINE', 'CHARITY', 'GROW-FA

In [5]:
params["WARMUP_STEPS"] = int(len(df) * params["EPOCHS"] * 0.1)  # 10% of train data
# df['snippet'] = df['replace_no_tags']

In [6]:
model_fit, test_evaluator, sbert_model = train(df, classifications, params, dataset_dir, sbert_model)


Training with one objective


KeyError: 'VAL_DEV_SIZE'

In [ ]:
sbert_model.evaluate(test_evaluator)

In [ ]:
frozen_model = load_model()
frozen_model.evaluate(test_evaluator)